In [1]:
import sklearn
import pandas as pd
import numpy as np
import pyodbc 
import re
from tqdm import tqdm_notebook as tqdm
#Load Data
startdate=20190801
enddate=20190831
conn = pyodbc.connect('DRIVER={SQL Server};SERVER=10.13.11.151;DATABASE=PBSA;UID=megaap;PWD=mega9701')
df=pd.read_sql_query("select * from vwactivity where bookdate between 20190801 and 20190831", conn)
#Only select following features
df=df[['TranNo','Bene', 'ByOrder', 'BookDate', 'BaseAmt', 'Cust','Account','PaymtMethod','Type','RecvPay']]
#only select wire transfer which is the type in 1001,1002,1003,1004
wire_txn_code=['1001','1002','1003','1004']
df=df[df['Type'].isin(wire_txn_code)]
#remove MT202 txn which is bank to bank txn
df=df[df['PaymtMethod']!='202']
print(df.columns.values)

['TranNo' 'Bene' 'ByOrder' 'BookDate' 'BaseAmt' 'Cust' 'Account'
 'PaymtMethod' 'Type' 'RecvPay']


In [2]:
df.head(3)

,TranNo,Bene,ByOrder,BookDate,BaseAmt,Cust,Account,PaymtMethod,Type,RecvPay
155,1164801,MACRON ASSOCIATE COMPANY,NUMBER EIGHT LIGHTING,20190801,32496.16,NYA000300150,NYAUSD175101010000300998,103,1002,1
156,1164800,LEE AND LI ATTORNEYS AT LAW,LINCOLN ELECTRIC COMPANY,20190801,11084.00,NYA000300150,NYAUSD175101010000300998,103,1002,1
157,1164799,TIMES PACIFIC LIMITED,THOMSON INTERNATIONAL INC,20190801,130052.60,NYA000300150,NYAUSD175101010000300998,202COV,1002,1


In [5]:
###########LDA#############
def Amt_bin(amt,bin_num,rankAmt):
        for idx, th in enumerate(rankAmt):
            if amt<=th:
                return idx
        return bin_num
    
bin_num=20
rankAmt=[df['BaseAmt'].quantile(i/bin_num) for i in range(bin_num)]
df['BaseAmt_Bin']=df['BaseAmt'].apply(Amt_bin,args=(bin_num,rankAmt))
df.head(3)

,TranNo,Bene,ByOrder,BookDate,BaseAmt,Cust,Account,PaymtMethod,Type,RecvPay,BaseAmt_Bin
155,1164801,MACRON ASSOCIATE COMPANY,NUMBER EIGHT LIGHTING,20190801,32496.16,NYA000300150,NYAUSD175101010000300998,103,1002,1,14
156,1164800,LEE AND LI ATTORNEYS AT LAW,LINCOLN ELECTRIC COMPANY,20190801,11084.00,NYA000300150,NYAUSD175101010000300998,103,1002,1,10
157,1164799,TIMES PACIFIC LIMITED,THOMSON INTERNATIONAL INC,20190801,130052.60,NYA000300150,NYAUSD175101010000300998,202COV,1002,1,18


In [8]:
#pseudo_high_frequency_words contains all Prefix suffix, etc of a company
pseudo_high_frequency_words=['LIMITED LIABILITY COMPANY', 'INCORPORATED', 'INTERNATIONAL', 'LIMITED', 'COMPANY', 'CORPORATION', 'PRIVATE', 'INC','INTL', 'CORP', 'LTD', 'LLC', 'L L C', 'PTE', 'PVT', 'CO']
def Normalization_Pseudo_Name(name):
    if name is not None:
        name=re.sub('^[0-9]/','/',name)
        name=re.sub('[^0-9a-zA-Z]+',' ',name)
        for word in pseudo_high_frequency_words:
            name=re.sub(' '+word+' ',' ',name)
            name=re.sub(' '+word,' ',name)
        name=re.sub(' ','',name)
        return name
    else:
        return None
df['Norm_Bene']=df['Bene'].apply(Normalization_Pseudo_Name)
df['Norm_ByOrder']=df['ByOrder'].apply(Normalization_Pseudo_Name)
df.head(3)

,TranNo,Bene,ByOrder,BookDate,BaseAmt,Cust,Account,PaymtMethod,Type,RecvPay,BaseAmt_Bin,Norm_Bene,Norm_ByOrder
155,1164801,MACRON ASSOCIATE COMPANY,NUMBER EIGHT LIGHTING,20190801,32496.16,NYA000300150,NYAUSD175101010000300998,103,1002,1,14,MACRONASSOCIATE,NUMBEREIGHTLIGHTING
156,1164800,LEE AND LI ATTORNEYS AT LAW,LINCOLN ELECTRIC COMPANY,20190801,11084.00,NYA000300150,NYAUSD175101010000300998,103,1002,1,10,LEEANDLIATTORNEYSATLAW,LINCOLNELECTRIC
157,1164799,TIMES PACIFIC LIMITED,THOMSON INTERNATIONAL INC,20190801,130052.60,NYA000300150,NYAUSD175101010000300998,202COV,1002,1,18,TIMESPACIFIC,THOMSON


In [72]:
#group byorder
g_byorder=df.groupby('Norm_ByOrder')
Byorder_group=[i for i in g_byorder.__iter__()]

#group bene
g_bene=df.groupby('Norm_Bene')
Bene_group=[i for i in g_bene.__iter__()]

#features used in byorder should exclude byorder name
feature_used_in_byorder=['Cust','Account',
       'PaymtMethod', 'Type', 'RecvPay', 'Norm_Bene','BaseAmt_Bin']

#words_generator()
def words_generator(df,feature_used_in_word):
        df=df.applymap(str)
        df['words']=df[feature_used_in_word].apply(lambda x: '_'.join(x),axis=1)
        return df['words'].values
    
#generate all words for each originator, return [(originator1,[word1,word2.....]),...]
Byorder_num=len(Byorder_group)
Byorder_words=[(Byorder_group[i][0], 
                words_generator(Byorder_group[i][1],
                feature_used_in_word)) for i in tqdm(range(Byorder_num))]

#features used in benefiiary should exclude byorder name
feature_used_in_bene=['Cust','Account',
       'PaymtMethod', 'Type', 'RecvPay', 'Norm_ByOrder','BaseAmt_Bin']

#generate all words for each beneficiary, return [(beneficiary1,[word1,word2.....]),...]
Bene_num=len(Bene_group)
Bene_words=[(Bene_group[i][0], 
                words_generator(Bene_group[i][1],
                feature_used_in_bene)) for i in tqdm(range(Bene_num))]

In [152]:
#create dataframe for originator and beneficiary
Byorder_words_df=pd.DataFrame(Byorder_words,columns=['name','words'])
Bene_words_df=pd.DataFrame(Bene_words,columns=['name','words'])

#merge two df
merge_word_df=Byorder_words_df.merge(Bene_words_df,how='outer',
                                     left_on='name',
                                     right_on='name',
                                     suffixes=['_byorder','_bene'])
#combine words with same name
merge_word_df['words_merge']=merge_word_df.apply(lambda x: np.append(x['words_byorder'],
                                                                     x['words_bene']),axis=1)
#delete nan from words_merge
merge_word_df['words_merge']=merge_word_df['words_merge'].apply(lambda x: x[~pd.isnull(x)])
#keep two columns only
merge_word_df=merge_word_df[['name','words_merge']]
merge_word_df.head(3)

,name,words_merge
0,1001DUSALP,[NYA000300150_NYAUSD175101010000300998_103_100...
1,100MEGADISTRIBUTIONSRO,[NYA000300588_NYAUSD175101010000300588_103_100...
2,1031,[NYA000300150_NYAUSD175101010000300998_103_100...


In [156]:
#########LDA#############
#generate vocabulary
def vocab_generator(Series):
    vocab=np.array([])
    for i in Series:
        vocab=np.append(vocab,i)
    return np.unique(vocab)

vocab=vocab_generator(merge_word_df['words_merge'])
vocab

array(['NYA000017434_NYAUSD208000000000017434_103_1001_1_MINISTRYOFFOREIGNAFFAIRSOF_13',
       'NYA000017434_NYAUSD208000000000017434_103_1001_1_MINISTRYOFFOREIGNAFFAIRSOF_7',
       'NYA000017434_NYAUSD208000000000017434_103_1001_1_MMERCIALOFFICEOFTHE_9',
       ...,
       'NYA100000339_NYAUSD208000000000053546_103_1001_1_MINISTRYOFFOREIGNAFFAIRSOF_6',
       'NYA100000357_NYAUSD208000000000061820_103_1001_1_CHIENHUNGSHENGCHUSUFEN_11',
       'NYA100000357_NYAUSD208000000000061820_103_1001_1_CHIENHUNGSHENG_11'],
      dtype=object)

In [162]:
#convert word to index
def word2idx(vocab,df):
        word2idx_dic={word:idx for idx,word in enumerate(vocab)}
        word2idx_dic['padding']=len(vocab)
        df['word2idx']=merge_word_df['words_merge'].apply(lambda doc:np.array([word2idx_dic[word] for word in doc]))
        return word2idx_dic

word2idx_dic=word2idx(vocab,merge_word_df)

In [164]:
merge_word_df

,name,words_merge,word2idx
0,1001DUSALP,[NYA000300150_NYAUSD175101010000300998_103_100...,"[5476, 5473, 5477]"
1,100MEGADISTRIBUTIONSRO,[NYA000300588_NYAUSD175101010000300588_103_100...,[10387]
2,1031,[NYA000300150_NYAUSD175101010000300998_103_100...,[4186]
3,1308072ONTARIODBAFACTO,[NYA000300150_NYAUSD175101010000300998_202COV_...,[6534]
4,1871WOODSLEETROYMI48083UNITED,[NYA000300150_NYAUSD175101010000300998_202COV_...,[6285]
...,...,...,...
6367,ZITAIPRECISIONMACHINERY,[NYA000300150_NYAUSD175101010000300998_202COV_...,[8130]
6368,ZM,[NYA000300150_NYAUSD175101010000300998_202COV_...,[9703]
6369,ZOEINDUSTRIAL,[NYA000300150_NYAUSD175101010000300998_202COV_...,"[6192, 2658]"
6370,ZOOMENTERPRISE,[NYA000300150_NYAUSD175101010000300998_103_100...,[2621]


In [166]:
tp=merge_word_df['word2idx'].apply(lambda x: len(x))
tp.value_counts()

1      4054
2      1167
3       419
4       266
5       133
6        83
7        58
8        32
9        25
10       24
11       17
13       10
14        8
12        8
15        7
22        5
18        5
16        5
17        4
21        3
23        3
25        3
40        2
44        2
26        2
29        2
20        2
19        2
59        2
57        2
75        1
35        1
31        1
36        1
288       1
41        1
37        1
497       1
83        1
38        1
46        1
54        1
58        1
70        1
86        1
150       1
219       1
Name: word2idx, dtype: int64